### import 및 전역변수

In [11]:
import pandas as pd
from tqdm import tqdm
import re, os
from os.path import join

account_dtype = {'채무자키':str, '계좌키':str, '타채무자키':str}
memo_dtype = {'채무자키' : str, '메모키':str}
event_dtype = {'채무자키':str, '법조치키':str, '계좌키':str, '관련법조치키':str, '법취하키':str, '타법조치키':str, '타채무자키':str, '관할법원코드':str}
deposit_dtype = {'채무자키' : str, '입금키':str, '계좌키':str, '입금계좌번호':str, '계좌번호':str, '타입금키':str, '타채무자키':str, '보증인키':str}

In [12]:
################################################################
company = "솔림"
cutDate = "230731"
sellDate = "230831"
cutDate_to_input = "2023-07-31"
sellDate_to_input = "2023-08-31"
sell_ext = ".pkl" # 매각일 전산자료 처음 받았다면 xlsx
path_save = r"D:\3.자산\POOL별 관리자산\솔림\POOL 1 번자료\매각\애플_신용회복\솔림_전산자료_애플_"+cutDate+".xlsx"
매각키정리한sheet_name = "P1애플(신복)230831"
################################################################

### 파일 읽기

In [13]:
# 파일읽기 # 매각일 자료도 일단은 dataset에 저장하고 나중에 옮기든지 하자.
cut_ext = ".pkl" # 자산정리하면서 pkl 만들었으니
path_base = join(r"D:\3.자산\전산 dataset", company)

cutFiles = [name + cut_ext for name in ["채무자조회새창_20230731_2106", "계좌조회새창_20230731_2109", "보증인새창_20230731_2110", \
    "개인회생새창_20230731_2115_진행중_계좌키분리_기준데이터", "20230731_신용회복전체리스트_솔림"]]
                                                            ###### 신복, 개회 기준데이터로 ###### 그래야 자산정리와 같고, 날짜랑도 맞는다.
sellFiles = ["계좌조회새창_20230831_2250", "메모조회새창_20230905_0933", "법조치조회새창_20230831_2254", "입금조회새창_20230905_0939"]

# 자산확정일
cut_dir = join(path_base, cutDate)
debt = pd.read_pickle(join(cut_dir, cutFiles[0]))
cut_accout = pd.read_pickle(join(cut_dir, cutFiles[1]))
grt = pd.read_pickle(join(cut_dir, cutFiles[2]))
rehabilitation = pd.read_pickle(join(cut_dir, cutFiles[3]))
credit = pd.read_pickle(join(cut_dir, cutFiles[4]))

# 매각일
sell_dir = join(path_base, sellDate)
if sell_ext == ".xlsx" : 
    sell_account = pd.read_excel(join(sell_dir, sellFiles[0]+sell_ext), dtype=account_dtype ).fillna("")
    memo = pd.read_excel(join(sell_dir, sellFiles[1]+sell_ext), dtype=memo_dtype ).fillna("")
    event = pd.read_excel(join(sell_dir, sellFiles[2]+sell_ext), dtype=event_dtype ).fillna("")
    deposit = pd.read_excel(join(sell_dir, sellFiles[3]+sell_ext), dtype=deposit_dtype ).fillna("")
else : # pkl
    sell_account = pd.read_pickle(join(sell_dir, sellFiles[0]+sell_ext))
    memo = pd.read_pickle(join(sell_dir, sellFiles[1]+sell_ext))
    event = pd.read_pickle(join(sell_dir, sellFiles[2]+sell_ext))
    deposit = pd.read_pickle(join(sell_dir, sellFiles[3]+sell_ext))

# 신복기준데이터의 No.열 없애기
credit = credit.drop(['No.'], axis=1)

# 매각리스트
keys = pd.read_excel(r"C:\Users\SL\Desktop\매각건정리.xlsx", sheet_name=매각키정리한sheet_name, dtype=str).fillna("")

In [ ]:
# # 매각일 자료 pkl 저장 -----------
# sell_account.to_pickle(join(sell_dir, sellFiles[0]+cut_ext))
# memo.to_pickle(join(sell_dir, sellFiles[1]+cut_ext))
# event.to_pickle(join(sell_dir, sellFiles[2]+cut_ext))
# deposit.to_pickle(join(sell_dir, sellFiles[3]+cut_ext))

### 데이터 추출

In [14]:
# merge(df-df)를 위해 키 중복값 제거
debtkeys = keys[["채무자키"]].drop_duplicates('채무자키') # df리턴
accountkeys = keys[["계좌키"]].drop_duplicates('계좌키') # df리턴 # 중복값 없겠지만 혹시 모르니
# 채무자키와 새채무자키
new_debtkeys = debt[["채무자키","관리자비고"]].merge(debtkeys, how="right").rename(columns = {"관리자비고":"새채무자키"})
print(len(debtkeys), len(accountkeys), len(new_debtkeys))

570 570 570


In [15]:
# 법조치 새채무자키 추가하기 (new_debekeys의 채무자키에 중복값이 없으므로 숫자변화 없다)
print(len(event))
event = event.merge(new_debtkeys, on='채무자키', how='left')
print(len(event))

156712
156712


In [17]:
# 추출하기 : 법조치는 새채무자키로 추출
# 자산확정일 : 채무자, 계좌, 보증인, 개인회생, 신용회복
# 매각일 : 계좌, 입금(계좌조회,계좌별표시,기간제한x), 메모, 법조치(새채무자키)
# 채무자 : 채무자키
debt = debt.merge(debtkeys, how='inner')
# 계좌_확정일 : 계좌키 - 계좌키 누락/일부 매각 없어야
cut_accout = cut_accout.merge(accountkeys, how='inner')
# 보증인 : 계좌키
grt = grt.merge(accountkeys, how='inner') # 보증인
# 개인회생 : 계좌키
rehabilitation = rehabilitation.merge(accountkeys, how='inner')
# 신용회복 : 계좌키
credit = credit.merge(accountkeys, how='inner')
# 법조치, 채무자키 기준 vs 새채무자키 기준 - 계좌키는 제대로 등록이 되어있지 않으므로 사용할 수 없다.
event_채무자키 = event.merge(debtkeys, how='inner', on='채무자키')
event = event.merge(new_debtkeys[['새채무자키']], how='inner', on='새채무자키')
print(f"법조치 - 채무자키로 법조치 추출 : {len(event_채무자키)}, 새채무자키로 법조치 추출 : {len(event)}")
# 메모 : 채무자키
memo = memo.merge(debtkeys, how='inner')
# 계좌_매각일
sell_account = sell_account.merge(accountkeys, how='inner')
# 입금 : 계좌키
deposit = deposit.merge(accountkeys, how='inner')

# 법조치 새채무자키든 채무자키든 결과차이 없다면 새채무자키 열 삭제 
if len(event_채무자키) == len(event) : 
    event.drop(['새채무자키'], axis=1)

법조치 - 채무자키로 법조치 추출 : 232, 새채무자키로 법조치 추출 : 232


### 칼럼 편집

In [ ]:
dfs = [debt, cut_accout, grt, rehabilitation, credit, event, memo, deposit, sell_account]
df_names = ["채무자", "계좌", "보증인", "개인회생", "신용회복", "법조치_"+sellDate, "memo_"+sellDate, "deposit_"+sellDate, "계좌_"+sellDate]

# # 문제되는 칼럼 살피기
# x = set()
# y = {"debt" : [], "cut_accout" : [], "grt" : [], "rehabilitation" : [], "credit" : [], "event" : [], "memo" : [], "deposit" : [], "sell_account" : []}
# for i, df in enumerate(dfs):
#     for col in df.columns :
#         if re.search('이자기준일', col) :
#             y.get(df_names[i]).append(col)
#             x.add(col)
# print(x)
# for k,v in y.items() : 
#     print(k, ":", v)

In [ ]:
# 컬럼 내용 편집
# 초기화할 열 목록
columns_to_reset = ['담당자','담당자키','담당핸드폰','송무담당', '담당입금구분', '담당문자전송', '담당연락처', '전담당자', '담당변경일', '지점','지역지점', # 채무상태
     '부서', '팀', '등록파일', '등록유저', '수정파일', '법조치파일','등록유저', '수정유저', '담보제공처', '담보제공구분', '채권매각일', '채권매각대금', '채권매각처', '채권환매대금','채권환매일',
     '채권손실일', '채권손실금액', '계약일', '입금은행', '상대방입금은행', '입금계좌번호', '입금순서']
columns_to_input = ["매입일", "컷오프일", "최초이자기준일"]

# 모든 데이터프레임에 대해 열 초기화
for df in dfs:
    for col in columns_to_reset: # 삭제
        if col in df.columns:
            df[col] = ""
    for ind, col in enumerate(columns_to_input): # 컷오프일 or 매입일 작성
        if col in df.columns:
            if ind==0 : df[col] = sellDate_to_input
            else : df[col] = cutDate_to_input

In [ ]:
# 메모시트 등록일자 편집 및 매각메모 삭제
############################################################
삭제할메모내용 = "23.08.31 매각(애플) 02-3474-0522"
############################################################
# 공지 기호 삭제
memo["등록일자"] = memo.등록일자.str.replace("★ | ★", "", regex=True)
# 매각 메모 삭제
before = len(memo)
memo = memo[~(memo.메모내용 == 삭제할메모내용)]
memo.reset_index(drop=True)
after = len(memo)
print(f"삭제한 메모수 : {before} - {after} = {before-after}, 매각한 채무자키 수 : {len(debtkeys)}")
# 등록일자순 정렬
memo = memo.sort_values(['등록일자'], ascending=False)
# 메모색상, 공지여부
memo['메모색상'] = "검정"
memo['공지여부'] = "N"

### 저장

In [ ]:
# 엑셀 저장 (파일이 오류나네.. 근데 내용은 문제 없어서, 복구 후 다른이름으로 저장하면 괜찮았다.)
# dfs에 변경된 내용 반영 (drop과 sort가 반영이 안 되네..)
dfs = [debt, cut_accout, grt, rehabilitation, credit, event, memo, deposit, sell_account]
df_names = ["채무자", "계좌", "보증인", "개인회생", "신용회복", "법조치_"+sellDate, "메모_"+sellDate, "입금_"+sellDate, "계좌_"+sellDate]
dict_sheet_df = dict(zip(df_names, dfs))

with pd.ExcelWriter(path_save, engine='openpyxl', mode = 'w') as writer:
    for sheetname, df_name in dict_sheet_df.items():
        df_name.to_excel(writer, sheet_name = sheetname, index=False)

In [ ]:
# readme - 다음을 복사
readme = """
시트명에 날짜가 없는 것은 컷오프일 기준의 전산데이터입니다.
시트명에 날짜가 붙은 것은 매각일 기준의 전산데이터입니다.

삭제정보 : 불필요한 정보(솔림내부정보 : 담당자, 입금계좌 등)는 삭제하였습니다.
수정정보 : 매입일, 컷오프일, 최초이자기준일은 금번 계약 내용으로 수정하여 두었습니다.
유지정보(등록시 수정필요) : 매각사, 각종 키값 및 타키값(타채무자키 등)은 일단 솔림 전산값 그대로 두었으니 전산 등록시 수정하시기 바랍니다.

유의할칼럼 : 
    계좌의 OPB - (매입당시)최초원금+최초법비용입니다. (현재값이 아닙니다.)
"""
#    법조치의 새채무자키 - 매각대상에 없는 채무자키가 들어있을 수 있습니다. 이는 채무자키는 다르지만, ....